In [1]:
using LinearAlgebra
using Statistics

In [121]:
# helper methods
function symetric_matrix(dim::Integer)::Matrix
    A = ones(dim,dim);
    for i = 1:dim
        for j = 1:dim
            A[i,j] = rand(1:50);
            A[j,i] = A[i,j];
        end
    end
    return A;
end

# we might change it to just rand(dim,dim)
function random_matrix(dim::Integer)::Matrix
    A = ones(dim, dim);
    for i = 1:dim
        for j = 1:dim
            A[i,j] = rand(1:50)
        end
    end
    return A
end

function diagonal_matrix(dim::Integer)::Matrix
    I = ones(dim,dim)
    return Diagonal(I)
end

function custom_cond(λₘₐₓ, λₘᵢₙ)
    return √(λₘₐₓ/λₘᵢₙ)
end

custom_cond (generic function with 1 method)

# Metoda potęgowa

Poniżej przygotowana jest funkcja do metody iteracji prostej (metoda potęgowa), dzięki której można wyznaczyć największą co do modułu wartość własną, czyli promień spektralny macierzy, który opisany jest wzorem:
## $$\rho(A) = |\lambda_{max}| = \lim_{i \to \infty} \frac{||t_{i+1}||_{\infty}}{||t_{i}||_{\infty}}$$

Przy założeniu, że dowolny wektor początkowy t_{0} != 0, do uzyskania promienia spektralnego prowadzą iteracje postaci:
## $$t_{i+1} = At_{i},\ i=0,1,2...$$



In [48]:
function power_eigen(A::Matrix, iterations::Integer)::Array
    n = size(A, 1);
    X = ones(n,1);
    for i = 1:iterations
        X = A * X;
        X = X / norm(X);
    end

    return X' * A * X / (X' * X);
end

function power_eigen_min(A::Matrix, iterations::Integer)::Array
    n = size(A, 1);
    X = ones(n,1);
    for i = 1:iterations
        X = A \ X;
        X = X / norm(X);
    end

    return X' * A * X / (X' * X);
end

power_eigen_min (generic function with 1 method)

In [78]:
A = symetric_matrix(5);
#A = [22 21 -8 4; 21 49 11 39; -8 11 60 -19; 4 39 -19 135]
@time λₘₐₓ = power_eigen(A, 10);
print(λₘₐₓ)
e, v = eigen(A)

  0.023384 seconds (38.70 k allocations: 1.988 MiB)
[148.21815636561124]

Eigen{Float64,Float64,Array{Float64,2},Array{Float64,1}}
values:
5-element Array{Float64,1}:
 -41.34773473241207
 -17.235379556224004
   7.200226098135396
  48.16473182487727
 148.21815636562317
vectors:
5×5 Array{Float64,2}:
  0.442534    0.490963   -0.298811  -0.475493   -0.497733
 -0.776593    0.419416    0.15139    0.0744663  -0.438782
 -0.181066   -0.686056    0.151079  -0.573519   -0.380516
  0.0199452  -0.334484   -0.605532   0.546892   -0.47113
  0.409737   -0.0222159   0.705903   0.374628   -0.43929

Jak widać z wyniku wykonania powyższego kodu, uzyskaliśmy największą co do modułu wartość własną macierzy. Ta wartość zgadza sięz wartością otrzymaną za pomocą metody eigen(A). Za pomocą metody potęgowej można również wyznaczyć najmniejszą co do modułu wartość własną macierzy. Korzysta się z twierdzenia dotyczącego przesunięcia spektrum macierzy:<br />
### "Jeżeli $\lambda$ jest wartością własną macierzy A, to $\lambda + r$ jest wartością własną macierzy $A + \tau I$"
Należy pamiętać, że twierdzenie ma zastosowanie dla macierzy symetrycznych i dodatnio określonych. Po zastosowaniu przesunięcia $B = A - \lambda_{max}I$ można ponownie zastosować metodę iteracji prostej dla macierzy B zbieżną do największej co do modułu wartości własnej $\lambda = \lambda_{min}-\lambda_{max}$. Stąd można wyznaczyć $\lambda_{min}$.

In [79]:
I = diagonal_matrix(size(A,1))
#B = C - λₘₐₓ .* I
#λ = power_eigen(B, 10);
#λₘᵢₙ = λ + λₘₐₓ
#print(λₘᵢₙ)
λₘᵢₙ = power_eigen_min(A,20)

1×1 Array{Float64,2}:
 7.200226098135385

Mając największą i najmniejszą wartość własną macierzy, można wyznaczyć współczynnik uwarunkowania macierzy:
$$cond(A)=\sqrt{\frac{\lambda_{max}(A^TA)}{\lambda_{min}(A^TA)}}$$
We wzorze przyjęto, że macierz A jest symetryczna, co oznacza, że $A=A^T$, ewentualnie można macierz wejściową doprowadzić do postaci symetrycznej stosując wzór $B = A^TA$

In [80]:
value = custom_cond(λₘₐₓ, λₘᵢₙ)
print(value)

√cond(A)


[4.537092529735147]

4.537092529735323

# Metoda Jacobiego

In [122]:
function jacobi_eigen(A::Matrix, iterations::Integer)
    n = size(A,1);
    for i = 1:iterations
        s,t = maxst(A);
        
        d = √((A[s,s] - A[t,t])^2 + 4 * A[s,t]^2);
        sin2t = 2 * A[s,t] / d;
        cos2t = (A[s,s] - A[t,t]) / d;
        dt = √(2 * (1 + cos2t));
        sint = abs(sin2t) / dt;
        cost = abs((1 + cos2t) / dt);
        cost = sign(A[s,t]) * cost;

        R = diagonal_matrix(n);
        R[s,s] = cost;
        R[t,t] = cost;
        R[s,t] = -sint;
        R[t,s] = sint;
        
        A = R' * A * R; 
        
    end
    return A
end

function maxst(A::Matrix)
    s = 1;
    t = 2;
    n = size(A,1);
    for c = 2:n
        for r = 1:c-1
            if abs(A[r,c]) > abs(A[s,t])
                s = r;
                t = c;
            end
        end
    end
    return s, t
end

maxst (generic function with 3 methods)

In [267]:
#F = [12 6 -6 1; 6 16 2 100; -6 2 16 1 ;1 100 1 4];
F = rand(4,4);
FJ = jacobi_eigen(F, 15);
FJ[abs.(FJ) .< 1e-10] .= 0.0;
display(FJ)
e,v = eigen(F);
display(e)


4×4 Array{Float64,2}:
  1.86449   -0.000603977  -0.00106943  -0.000572329
 -0.726941   0.360242     -0.0489468    0.00517663
  0.244309   0.116424      0.161888     0.00497935
  0.678177  -0.194209     -0.191085    -0.287318

4-element Array{Float64,1}:
 -0.2837619520283921
  0.19643766676891092
  0.3221619923789694
  1.8644618064886556